# Libratires

In [ ]:
# !pip -q install --user pip==25.0.1

In [ ]:
# !pip install --user -q mlflow datasets>=2.14.5 nlp 2>/dev/null
# !pip install --user -q accelerate -U
# !pip install --user -q transformers

# !pip install --user -q peft==0.6.2
# !pip install --user -q --upgrade torch
!pip install --user -q gdown

In [ ]:
# !pip uninstall --user -y torch torchvision transformers
# !pip uninstall -y kaggle-environments pytorch-lightning sentence-transformers torchaudio
# !pip -q install --user torch==2.0.1 torchvision==0.15.2 --no-cache-dir
# !pip -q install fastai==2.7.18

In [ ]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import gc  # For garbage collection to manage memory
import re  # For regular expressions
import numpy as np
import time

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    roc_auc_score,
    recall_score,
    classification_report,
    f1_score
)

import torch
from torch import nn
from transformers import (AutoModel,
                          Trainer,
                          TrainingArguments,
                          AutoTokenizer,
                          DebertaV2Tokenizer,
                          DebertaV2Config)
from transformers import ( DataCollatorForLanguageModeling,
                         pipeline,
                         Trainer,
                         TrainingArguments,
                         DataCollatorWithPadding)
from transformers import (AutoModelForSequenceClassification,
                         BertForSequenceClassification,
                         RobertaForSequenceClassification)


import datasets
from datasets import Dataset, Image, ClassLabel
from transformers import pipeline

import matplotlib.pyplot as plt
import itertools
import gdown


from tqdm import tqdm
tqdm.pandas()

# Data installation

In [ ]:
!gdown --folder "https://drive.google.com/drive/folders/11YeloR2eTXcTzdwI04Z-M2QVvIeQAU6-"

Retrieving folder contents
Processing file 1oh9c-d0fo3NtETNySmCNLUc6H1j4dSWE subtaskB_dev.jsonl
Processing file 1k5LMwmYF7PF-BzYQNE2ULBae79nbM268 subtaskB_train.jsonl
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1oh9c-d0fo3NtETNySmCNLUc6H1j4dSWE
To: /content/SubtaskB/subtaskB_dev.jsonl
100% 4.93M/4.93M [00:00<00:00, 36.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1k5LMwmYF7PF-BzYQNE2ULBae79nbM268
From (redirected): https://drive.google.com/uc?id=1k5LMwmYF7PF-BzYQNE2ULBae79nbM268&confirm=t&uuid=c76611af-e560-45b2-a5c8-6e9a555249a5
To: /content/SubtaskB/subtaskB_train.jsonl
100% 155M/155M [00:01<00:00, 127MB/s]
Download completed


In [ ]:
input_path = "/content/SubtaskB/subtaskB_train.jsonl"
df = pd.read_json(input_path, lines=True)
df.head()

,text,model,source,label,id
0,Forza Motorsport is a popular racing game that...,chatGPT,wikihow,1,0
1,Buying Virtual Console games for your Nintendo...,chatGPT,wikihow,1,1
2,Windows NT 4.0 was a popular operating system ...,chatGPT,wikihow,1,2
3,How to Make Perfume\n\nPerfume is a great way ...,chatGPT,wikihow,1,3
4,How to Convert Song Lyrics to a Song'\n\nConve...,chatGPT,wikihow,1,4


In [ ]:
df = df[df.model != 'bloomz']
df = df[df.model != 'dolly']
df = df.drop(['source'], axis=1)
df['human_ai_labels'] = df['model'].apply(lambda x: 0 if x == 'human' else 1)
df = df.rename(columns={'label': 'ai_model_labels'})
print(df.shape)
df.head()

(47327, 5)


,text,model,ai_model_labels,id,human_ai_labels
0,Forza Motorsport is a popular racing game that...,chatGPT,1,0,1
1,Buying Virtual Console games for your Nintendo...,chatGPT,1,1,1
2,Windows NT 4.0 was a popular operating system ...,chatGPT,1,2,1
3,How to Make Perfume\n\nPerfume is a great way ...,chatGPT,1,3,1
4,How to Convert Song Lyrics to a Song'\n\nConve...,chatGPT,1,4,1


In [ ]:
def class_weights(df):
    # Identify the unique classes in the training data.
    class_names = np.unique(df[['model']])

    # Calculate class weights using the 'balanced' option, which automatically adjusts for class imbalance.
    weights_imb = compute_class_weight(class_weight=None, classes=class_names, y=df['model'])

    # Create a dictionary mapping each class to its respective class weight.
    class_weights_imb = dict(zip(class_names, weights_imb))

    # Print the computed class weights to the console.
    print(class_weights_imb)

In [ ]:
proportion = 0.5

# Calculate the number of samples for each class in the smaller DataFrame
n_human = int(len(df[df['human_ai_labels'] == 0]) * proportion)
n_ai = int(len(df[df['human_ai_labels'] == 1]) * proportion)

# Get a stratified sample for each class
human_samples = df[df['human_ai_labels'] == 0].sample(n=n_human, random_state=42)
ai_samples = df[df['human_ai_labels'] == 1].sample(n=n_ai, random_state=42)

# Concatenate the samples to create the smaller DataFrame
df_smaller = pd.concat([human_samples, ai_samples])

# Shuffle the smaller DataFrame (optional)
df_smaller = df_smaller.sample(frac=1, random_state=42).reset_index(drop=True)

print(df_smaller.shape)
df_smaller.head()

(23663, 5)


,text,model,ai_model_labels,id,human_ai_labels
0,"The Achaemenid Persian Empire, also known as ...",cohere,2,70555,1
1,"Witnessing to a Mormon can be a challenge, but...",chatGPT,1,766,1
2,During the Renaissance and the Napoleonic War...,cohere,2,69734,1
3,Make sure you actually close out the account....,human,0,2487,0
4,Lavalius Cyrone Gordon (born 1937) is an Ameri...,human,0,19449,0


In [ ]:
class_weights(df_smaller)

{'chatGPT': np.float64(1.0), 'cohere': np.float64(1.0), 'davinci': np.float64(1.0), 'human': np.float64(1.0)}


In [ ]:
class_weights(df)

{'chatGPT': np.float64(1.0), 'cohere': np.float64(1.0), 'davinci': np.float64(1.0), 'human': np.float64(1.0)}


# Prepocessing

In [ ]:
# Fraction of the dataset used for training, the rest will be used for validation
train_fraction = 0.9

# Pre-trained BERT model to be used
BERT_MODEL = "microsoft/deberta-v3-base"

In [ ]:
# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Split into train/test (if not already split)
dataset = dataset.train_test_split(test_size=1 - train_fraction, seed=42)

In [ ]:
# tokenizer = DebertaV2Tokenizer.from_pretrained(BERT_MODEL)
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL,
                                          use_fast=True,
                                          low_cpu_mem_usage=False)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    tokenized["human_ai_labels"] = torch.tensor(examples["human_ai_labels"])
    tokenized["ai_model_labels"] = torch.tensor(examples["ai_model_labels"])
    return tokenized

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/42594 [00:00<?, ? examples/s]

Map:   0%|          | 0/4733 [00:00<?, ? examples/s]

# Model with only Multitask

In [ ]:
# Download the Python file using wget
!wget -O multitask_model.py https://raw.githubusercontent.com/AnnaRemi/Article_2025/refs/heads/main/multitask_classification_model.py


--2025-05-17 12:53:33--  https://raw.githubusercontent.com/AnnaRemi/Article_2025/refs/heads/main/multitask_classification_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4454 (4.3K) [text/plain]
Saving to: ‘multitask_model.py’

multitask_model.py  100%[===================>]   4.35K  --.-KB/s    in 0s      

2025-05-17 12:53:33 (59.6 MB/s) - ‘multitask_model.py’ saved [4454/4454]



In [ ]:
from multitask_model import DebertaV2ForAIDetection

config = DebertaV2Config.from_pretrained(BERT_MODEL)

# Initialize model (4 classes for AI models in your case)
model = DebertaV2ForAIDetection(config, num_ai_models=4)

In [ ]:
# Clear any cached memory
torch.cuda.empty_cache()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
model.print_layer_info()


Model Architecture Info:
- Total layers in DeBERTa encoder: 12
  Layer 1/12: Trainable
  Layer 2/12: Trainable
  Layer 3/12: Trainable
  Layer 4/12: Trainable
  Layer 5/12: Trainable
  Layer 6/12: Trainable
  Layer 7/12: Trainable
  Layer 8/12: Trainable
  Layer 9/12: Trainable
  Layer 10/12: Trainable
  Layer 11/12: Trainable
  Layer 12/12: Trainable

- Trainable layers: 12/12
- Trainable heads: Human-AI and 4-way AI classifier


In [ ]:
class MultitaskTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = {
            "human_ai_labels": inputs.pop("human_ai_labels"),
            "ai_model_labels": inputs.pop("ai_model_labels")
        }

        outputs = model(**inputs, **labels)
        loss = outputs["loss"]

        return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Unpack logits correctly
    print(type(logits), type(labels))

    human_ai_logits, ai_model_logits = logits
    human_ai_labels, ai_model_labels = labels

    # Compute predictions
    human_ai_preds = (human_ai_logits > 0).astype(int)  # Convert logits to binary predictions
    ai_model_preds = np.argmax(ai_model_logits, axis=1)  # Multi-class predictions

    # Metrics for binary classification (human vs. AI)
    human_ai_metrics = {
        # "human_ai_accuracy" : accuracy_score(human_ai_labels, human_ai_preds),
        "human_ai_f1": f1_score(human_ai_labels, human_ai_preds),
        "human_ai_precision": precision_score(human_ai_labels, human_ai_preds),
        "human_ai_recall": recall_score(human_ai_labels, human_ai_preds),
    }

    # Metrics for AI model classification (only for AI-generated texts)
    ai_indices = (human_ai_labels == 1)
    if ai_indices.any():
        ai_model_metrics = {
            # "ai_model_accuracy": accuracy_score(ai_model_labels[ai_indices], ai_model_preds[ai_indices]),
            "ai_model_f1": f1_score(ai_model_labels[ai_indices], ai_model_preds[ai_indices], average='macro'),
            "ai_model_precision": precision_score(ai_model_labels[ai_indices], ai_model_preds[ai_indices], average='macro'),
            "ai_model_recall": recall_score(ai_model_labels[ai_indices], ai_model_preds[ai_indices], average='macro'),
        }
    else:
        ai_model_metrics = {
            # "ai_model_accuracy": 0.0,
            "ai_model_f1": 0.0,
            "ai_model_precision": 0.0,
            "ai_model_recall": 0.0,
        }

    return {**human_ai_metrics, **ai_model_metrics}


In [ ]:
!rm -rf ~/.cache/huggingface

In [ ]:
import wandb
wandb.login(key="c06ac50290e8e6c909edcdb9edebdf57eb6d9580")

## Frozen 9 layers

In [ ]:
# Initialize model (4 classes for AI models in your case)
model = DebertaV2ForAIDetection(config, num_ai_models=4)

# Clear any cached memory
torch.cuda.empty_cache()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
model.unfreeze_top_layers(3)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="deberta_unfrozen3_5epochs_final",
    eval_strategy="epoch",
    num_train_epochs=5,
    logging_strategy="steps",
    logging_steps=500,
    report_to="wandb",
    disable_tqdm=False,
    save_total_limit=1,
)

# Initialize trainer
trainer = MultitaskTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)


start_time = time.time()
# Train the model
trainer.train()  # Full model training
training_time = time.time() - start_time
print(training_time)

# Save the final model
trainer.save_model("/kaggle/working/model_unfrozen")
wandb.finish()

Epoch,Training Loss,Validation Loss


## Frozen 6 layers

In [ ]:
# Initialize model (4 classes for AI models in your case)
model = DebertaV2ForAIDetection(config, num_ai_models=4)

# Clear any cached memory
torch.cuda.empty_cache()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
model.unfreeze_top_layers(6)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="deberta_unfrozen6_5epochs_final",
    eval_strategy="epoch",
    num_train_epochs=5,
    logging_strategy="steps",
    logging_steps=100,
    report_to="wandb",
    disable_tqdm=False,
    save_total_limit=1,
)

# Initialize trainer
trainer = MultitaskTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

start_time = time.time()
# Train the model
trainer.train()  # Full model training
training_time = time.time() - start_time
print(training_time)


# Save the final model
trainer.save_model("/kaggle/working/model")
wandb.finish()

# Model with LoRA & Multitask

## Data

In [ ]:
# Fraction of the dataset used for training, the rest will be used for validation
train_fraction = 0.9

# Pre-trained BERT model to be used
BERT_MODEL = "microsoft/deberta-v3-base"

In [ ]:
proportion = 0.5

# Calculate the number of samples for each class in the smaller DataFrame
n_human = int(len(df[df['human_ai_labels'] == 0]) * proportion)
n_ai = int(len(df[df['human_ai_labels'] == 1]) * proportion)

# Get a stratified sample for each class
human_samples = df[df['human_ai_labels'] == 0].sample(n=n_human, random_state=42)
ai_samples = df[df['human_ai_labels'] == 1].sample(n=n_ai, random_state=42)

# Concatenate the samples to create the smaller DataFrame
df_smaller = pd.concat([human_samples, ai_samples])

# Shuffle the smaller DataFrame (optional)
df_smaller = df_smaller.sample(frac=1, random_state=42).reset_index(drop=True)

print(df_smaller.shape)
df_smaller.head()

In [ ]:
# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Split into train/test (if not already split)
dataset = dataset.train_test_split(test_size=1 - train_fraction, seed=42)

tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL,
                                          use_fast=True,
                                          low_cpu_mem_usage=False)

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    tokenized["human_ai_labels"] = torch.tensor(examples["human_ai_labels"])
    tokenized["ai_model_labels"] = torch.tensor(examples["ai_model_labels"])
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)

## Model

In [ ]:
# Pre-trained BERT model to be used
BERT_MODEL = "microsoft/deberta-v3-base"

In [ ]:
!wget -O multitask_lora_model.py "https://raw.githubusercontent.com/AnnaRemi/Article_2025/refs/heads/main/multitask_lora_model.py"

In [ ]:
from multitask_lora_model import DebertaV2ForAIDetectionWithLoRA

config = DebertaV2Config.from_pretrained(BERT_MODEL,
                                        torch_dtype=torch.float32)

model_lora4 = DebertaV2ForAIDetectionWithLoRA(config,
                                       num_ai_models=4)

In [ ]:
model_lora4.print_layer_info()

In [ ]:
# Clear any cached memory
torch.cuda.empty_cache()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_lora4 = model_lora4.to(device)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np


def compute_metrics_lora(eval_pred):
    logits, labels = eval_pred

    human_ai_logits, ai_model_logits = logits
    human_ai_labels, ai_model_labels = labels


    # Binary task metrics (human vs AI)
    human_ai_preds = (human_ai_logits > 0).astype(int)

    binary_metrics = {
        # "human_ai_accuracy": accuracy_score(human_ai_labels, human_ai_preds),
        "human_ai_f1": f1_score(human_ai_labels, human_ai_preds, average='binary'),
        "human_ai_precision": precision_score(human_ai_labels, human_ai_preds, average='binary'),
        "human_ai_recall": recall_score(human_ai_labels, human_ai_preds, average='binary'),
    }

    # Multiclass task metrics (AI model detection)
    ai_model_preds = np.argmax(ai_model_logits, axis=1)

    # Only calculate AI model metrics for samples that are actually AI-generated
    ai_indices = (human_ai_labels == 1)

    metrics = binary_metrics

    if ai_indices.any():
        ai_model_metrics = {
            # "ai_model_accuracy": accuracy_score(ai_model_labels[ai_indices], ai_model_preds[ai_indices]),
            "ai_model_f1": f1_score(ai_model_labels[ai_indices], ai_model_preds[ai_indices], average='macro'),
            "ai_model_precision": precision_score(ai_model_labels[ai_indices], ai_model_preds[ai_indices], average='macro'),
            "ai_model_recall": recall_score(ai_model_labels[ai_indices], ai_model_preds[ai_indices], average='macro'),
        }
        metrics.update(ai_model_metrics)
    else:
        # If no AI samples in this batch, set metrics to 0 or some default
        metrics.update({
            # "ai_model_accuracy": 0.0,
            "ai_model_f1": 0.0,
            "ai_model_precision": 0.0,
            "ai_model_recall": 0.0,
        })

    return metrics

## Train 9 frozen layers

In [ ]:
model_lora4.unfreeze_top_layers(3)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="deberta_lora_unfrozen3_5epochs_final",
    eval_strategy="epoch",
    num_train_epochs=5,
    logging_strategy="steps",
    logging_steps=500,
    report_to="wandb",
    disable_tqdm=False,
    save_total_limit=1,
)

# Initialize trainer
trainer = MultitaskTrainer(
    model=model_lora4,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics_lora,
)

start_time = time.time()
# Train the model
trainer.train()  # Full model training
training_time = time.time() - start_time
print(training_time)

# Save the final model
trainer.save_model("/kaggle/working/model_lora")
wandb.finish()

## Train 6 frozen layers

In [ ]:
from multitask_lora_model import DebertaV2ForAIDetectionWithLoRA

config = DebertaV2Config.from_pretrained(BERT_MODEL,
                                        torch_dtype=torch.float32)

model_lora4 = DebertaV2ForAIDetectionWithLoRA(config,
                                       num_ai_models=4)

In [ ]:
model_lora4.unfreeze_top_layers(6)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="deberta_lora_unfrozen6_5epochs_final",
    eval_strategy="epoch",
    num_train_epochs=5,
    logging_strategy="steps",
    logging_steps=500,
    report_to="wandb",
    disable_tqdm=False,
    save_total_limit=1,
)

# Initialize trainer
trainer = MultitaskTrainer(
    model=model_lora4,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics_lora,
)

start_time = time.time()
# Train the model
trainer.train()  # Full model training
training_time = time.time() - start_time
print(training_time)

# Save the final model
trainer.save_model("/kaggle/working/model_lora")
wandb.finish()

# Thesis experiment - DistilRoBERTa

In [ ]:
import pandas as pd
import gc  # For garbage collection to manage memory
import re  # For regular expressions
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from peft import LoraConfig, TaskType
from peft import get_peft_model
import evaluate

import torch
from torch import nn
from transformers import AutoModel, AutoTokenizer
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification, RobertaForSequenceClassification

from imblearn.over_sampling import RandomOverSampler
import datasets
from datasets import Dataset, Image, ClassLabel
from transformers import pipeline
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    f1_score
)

# from datasets import load_metric
from evaluate import load

from tqdm import tqdm
tqdm.pandas()

## Data

In [ ]:
input_path = "/content/SubtaskB/subtaskB_train.jsonl"
df = pd.read_json(input_path, lines=True)
df.head()

df = df[df.model != 'bloomz']
df = df[df.model != 'dolly']
df = df.drop(['source'], axis=1)
df['human_ai_labels'] = df['model'].apply(lambda x: 0 if x == 'human' else 1)
df = df.rename(columns={'label': 'ai_model_labels'})
print(df.shape)
df.head()

(47327, 5)


,text,model,ai_model_labels,id,human_ai_labels
0,Forza Motorsport is a popular racing game that...,chatGPT,1,0,1
1,Buying Virtual Console games for your Nintendo...,chatGPT,1,1,1
2,Windows NT 4.0 was a popular operating system ...,chatGPT,1,2,1
3,How to Make Perfume\n\nPerfume is a great way ...,chatGPT,1,3,1
4,How to Convert Song Lyrics to a Song'\n\nConve...,chatGPT,1,4,1


In [ ]:
df = df[["text", "human_ai_labels"]].rename(columns={"human_ai_labels": "label"})
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)

## Model

In [ ]:
BERT_MODEL = 'distilroberta-base'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize(example):
    return tokenizer(example["text"],
                     truncation=True,
                     padding="max_length",
                     max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/42594 [00:00<?, ? examples/s]

Map:   0%|          | 0/4733 [00:00<?, ? examples/s]

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)

    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="binary")["recall"],
        "f1": f1.compute(predictions=preds, references=labels, average="binary")["f1"],
    }

## Train

In [ ]:
import wandb
wandb.login(key="c06ac50290e8e6c909edcdb9edebdf57eb6d9580")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: remizova-annn to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./distilroberta_results",
    run_name="distilroberta_binary",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_strategy="steps",
    logging_steps=500,
    report_to="wandb",
    disable_tqdm=False,
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.038600,0.009362,0.998098,0.998058,0.999444,0.998751
2,0.013700,0.027862,0.995986,0.995846,0.998889,0.997365
3,0.013200,0.010656,0.997465,0.997780,0.998889,0.998334
4,0.003400,0.022746,0.997042,0.996401,0.999722,0.998059
5,0.000000,0.024515,0.997676,0.997505,0.999444,0.998474


TrainOutput(global_step=26625, training_loss=0.016221981183651993, metrics={'train_runtime': 5066.9632, 'train_samples_per_second': 42.031, 'train_steps_per_second': 5.255, 'total_flos': 2.821158189176832e+16, 'train_loss': 0.016221981183651993, 'epoch': 5.0})

## LoRA

In [ ]:
from peft import LoraConfig, TaskType
from peft import get_peft_model

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
target_modules_names = ["query", "value"]
lora_config = LoraConfig(
    r=8, # LoRA attention dimension
    lora_alpha=16, # Alpha parameter for LoRA scaling
    target_modules=target_modules_names, # Modules to apply LoRA to (typical for attention layers)
    lora_dropout=0.1, # Dropout probability for LoRA layers
    bias="none", # Bias type for LoRA
    task_type="SEQ_CLS" # Task type
)

model_lora = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./distilroberta_lora_results",
    run_name="distilroberta_lora_binary",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_strategy="steps",
    logging_steps=500,
    report_to="wandb",
    disable_tqdm=False,
    save_total_limit=1,
)

trainer = Trainer(
    model=model_lora,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000600,0.011286,0.998098,0.997782,0.999722,0.998751
2,0.006300,0.009639,0.998310,0.997783,1.000000,0.998890
3,0.002900,0.010322,0.998098,0.997782,0.999722,0.998751
4,0.003700,0.010645,0.998310,0.997783,1.000000,0.998890
5,0.000300,0.011843,0.998310,0.997783,1.000000,0.998890


TrainOutput(global_step=26625, training_loss=0.0036900803153643584, metrics={'train_runtime': 3588.4859, 'train_samples_per_second': 59.348, 'train_steps_per_second': 7.42, 'total_flos': 2.869545147641856e+16, 'train_loss': 0.0036900803153643584, 'epoch': 5.0})